# Import

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer


# Data exploring

In [2]:
#data exploring
df = pd.read_csv('DATI.csv')

df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"['sotto', 'fermo', 'fermo', 'fermo', 'sinistra..."
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"['sotto', 'indietro', 'fermo', 'fermo', 'sinis..."
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"['fermo', 'indietro', 'fermo', 'fermo', 'sinis..."
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"['fermo', 'indietro', 'fermo', 'fermo', 'fermo..."
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"['fermo', 'indietro', 'fermo', 'fermo', 'fermo..."
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"['sopra', 'indietro', 'destra', 'sinistra brac..."
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"['fermo', 'indietro', 'destra', 'sinistra brac..."
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"['fermo', 'indietro', 'destra', 'sinistra brac..."
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"['fermo', 'indietro', 'destra', 'fermo', 'ferm..."


# Data cleaning

In [3]:
# rimuoviamo le label in eccesso tenendo solo le prime 3 righe
def trova_terza(str):
    for _ in range(3):
        idx = str.rfind(',')
        str = str[:idx]
    str = str + ']'
    print(str)

    return str

In [4]:
df['Label'] = df['Label'].apply(trova_terza)

['sotto', 'fermo', 'fermo']
['sotto', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['sopra', 'indietro', 'fermo']
['sopra', 'indietro', 'fermo']
['sopra', 'fermo', 'fermo']
['sopra', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietr

# aggregamento dati

In [5]:

def aggrega(df, colonna, batch_size=12):
    """
    Assegna la label più frequente ogni blocco di 10
    da specificare il df e la ['']
    """
    
    lista = []  
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]  
        
        labels = batch[colonna]  #label del batch
        counter = Counter(labels)     #trova la label più frequente 
        most_common_label = counter.most_common(1)[0][0]
        
        lista.extend([most_common_label] * len(batch))
    
    return pd.Series(lista, index=df.index)

In [6]:
df['Label'] = aggrega(df, colonna='Label')

In [7]:
df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"['fermo', 'indietro', 'fermo']"
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"['fermo', 'indietro', 'fermo']"
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"['fermo', 'indietro', 'fermo']"
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"['fermo', 'indietro', 'fermo']"
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"['fermo', 'indietro', 'fermo']"
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"['sotto', 'indietro', 'sinistra']"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"['fermo', 'indietro', 'destra']"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"['fermo', 'indietro', 'destra']"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"['fermo', 'indietro', 'destra']"


# Data cleaning PT 2

In [8]:
# la label è salvata come una stringa unica, quindi la puliamo e dividiamo con strip
# che dividerà in elementi di una lista
df['Label'] = df['Label'].apply(lambda x: x.replace('[',''))
df['Label'] = df['Label'].apply(lambda x: x.replace(']',''))
df['Label'] = df['Label'].apply(lambda x: x.replace('\'',''))

df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"fermo, indietro, fermo"
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"fermo, indietro, fermo"
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"fermo, indietro, fermo"
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"fermo, indietro, fermo"
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"fermo, indietro, fermo"
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"sotto, indietro, sinistra"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"fermo, indietro, destra"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"fermo, indietro, destra"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"fermo, indietro, destra"


In [9]:
df['Label'] = df['Label'].apply(lambda x: x.split(','))
df['Label']
#trasfroma in lista

0           [fermo,  indietro,  fermo]
1           [fermo,  indietro,  fermo]
2           [fermo,  indietro,  fermo]
3           [fermo,  indietro,  fermo]
4           [fermo,  indietro,  fermo]
                     ...              
40379    [sotto,  indietro,  sinistra]
40380      [fermo,  indietro,  destra]
40381      [fermo,  indietro,  destra]
40382      [fermo,  indietro,  destra]
40383      [fermo,  indietro,  destra]
Name: Label, Length: 40384, dtype: object

In [10]:
# come scelta progettuale ho eliminato le label non necessarie:

""" 
dato che facciamo una classificazione multi-classe dobbiamo dividere tutte le label e non ci serve sapere che dei
sensori sono in stato fermo, ma solo che il moviemento che ci interessa, quindi avanti/indietro/destra/ecc...
"""
def trasforma(lista):
    contatore = 0
    for elemento in lista:
        if elemento.strip() == 'fermo':
            lista.remove(elemento)
            contatore += 1
    
    if contatore == 3:
        elemento = 'fermo'
        return elemento
    
    return lista

In [11]:
df['Label'] = df['Label'].apply(trasforma)
df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,[ indietro]
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,[ indietro]
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,[ indietro]
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,[ indietro]
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,[ indietro]
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"[sotto, indietro, sinistra]"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"[ indietro, destra]"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"[ indietro, destra]"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"[ indietro, destra]"


In [12]:
df['Label'].value_counts()
#controllo se ci sono label vuote (no)

Label
[ fermo]                         7824
[ indietro]                      6948
[sopra,  indietro]               3252
[sotto,  indietro,  sinistra]    2880
[sopra,  fermo]                  2448
[ indietro,  sinistra]           2280
[sotto,  indietro]               2136
[ indietro,  destra]             1816
[ fermo,  sinistra]              1776
[sotto,  fermo]                  1668
[ avanti]                        1200
[sotto,  indietro,  destra]      1056
[sopra,  indietro,  destra]       984
[sopra,  indietro,  sinistra]     780
[sotto,  sinistra]                624
[sopra,  avanti]                  456
[sopra,  avanti,  sinistra]       396
[ fermo,  destra]                 348
[ avanti,  sinistra]              312
[sopra,  sinistra]                300
[sotto,  avanti,  sinistra]       264
[sotto,  avanti]                  132
[sotto,  destra]                  132
[sopra,  avanti,  destra]         120
[sopra,  destra]                  120
[ avanti,  destra]                 84
[sotto

In [13]:
# trasformazione label in binarie, per poi fornirle al modello
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(df['Label'])
print(binary_labels[0])

[0 0 0 1 0 0 0]


In [14]:
#mostra le classi corrispondenti
print(mlb.classes_) 

[' avanti' ' destra' ' fermo' ' indietro' ' sinistra' 'sopra' 'sotto']


# aggiuntiva per migliorare il training
non runnate a meno che sicuri

In [15]:
df2 = df.drop('timestamp', axis = 1)
df2 = df2.drop('gyroX', axis = 1)
df2 = df2.drop('gyroY', axis = 1)
df2 = df2.drop('gyroZ', axis = 1)


In [16]:
df2

,accX,accY,accZ,Label
0,-5.527879,-4.858718,-2.166334,[ indietro]
1,-5.200716,-5.413599,-2.259820,[ indietro]
2,-3.524709,-5.594171,-1.247677,[ indietro]
3,-0.023709,-6.387939,1.155626,[ indietro]
4,2.153291,-6.756805,2.129320,[ indietro]
...,...,...,...,...
40379,6.151457,-13.349115,13.103255,"[sotto, indietro, sinistra]"
40380,3.876457,-8.463914,11.822385,"[ indietro, destra]"
40381,2.689457,-6.649928,11.716661,"[ indietro, destra]"
40382,0.434457,-7.581928,12.356489,"[ indietro, destra]"


# train test split

In [17]:
# Caricamento e suddivisione dei dati
x = df2.drop('Label', axis  = 1)
y = binary_labels

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# normalizzazione

In [19]:
scaler = MinMaxScaler(feature_range=(0, 1))
x_train  = scaler.fit_transform(x_train)
x_test  = scaler.transform(x_test)

In [20]:
print(x_train.shape)
y_train.shape

(32307, 3)


(32307, 7)

# convertire i dati per il modello

In [21]:
sequence_length = 12 #sequenza degli istanti temporali

def create_sequences(train, sequence_length):
    
    seq_temp = []
    for i in range(len(train) - sequence_length + 1):
        seq_temp.append(train[i:i + sequence_length])
    return np.array(seq_temp)

x_train = create_sequences(x_train, sequence_length)
x_test = create_sequences(x_test, sequence_length)

print(x_train.shape)
x_test.shape

(32296, 12, 3)


(8066, 12, 3)

In [22]:
#scartiamo da y_test i dati in eccesso che non riescono a creare una sequenza intera
diff = len(y_test) - len(x_test)
print(diff)

if diff > 0:
    y_test = y_test[:-diff]

11


# addestramento

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
#modello
model = Sequential()
model.add(LSTM(256, input_shape=(sequence_length, 3), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(7, activation='softmax'))

early_stopping = EarlyStopping(
    monitor='val_loss',             #controlla la perdita della validazione
    patience=2,                     #numero di epoche senza miglioramenti dopo le quali fermare l'addestramento
    restore_best_weights=True       #ripristina i pesi del modello alla migliore epoca quando si ferma
)

# Compilazione del modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Allenamento
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks= early_stopping)



c:\Users\NicoloCalcagno\miniconda3\envs\UFS12DL\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


KeyboardInterrupt: 

In [26]:
from sklearn.metrics import accuracy_score
# Valutazione

predizioni = model.predict(x_test)
soglia_corretto = 0.6
predicted_labels = (predizioni > soglia_corretto)

accuracy = accuracy_score(y_test, predicted_labels)

253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step


In [27]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.3388296553434168


In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted_labels, target_names=mlb.classes_))


              precision    recall  f1-score   support

      avanti       0.71      0.30      0.42       602
      destra       0.00      0.00      0.00       940
       fermo       0.78      0.80      0.79      2812
    indietro       0.89      0.85      0.87      4417
    sinistra       0.70      0.32      0.44      1919
       sopra       0.00      0.00      0.00      1771
       sotto       0.00      0.00      0.00      1785

   micro avg       0.83      0.48      0.61     14246
   macro avg       0.44      0.32      0.36     14246
weighted avg       0.56      0.48      0.50     14246
 samples avg       0.76      0.55      0.62     14246



c:\Users\NicoloCalcagno\miniconda3\envs\UFS12DL\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\NicoloCalcagno\miniconda3\envs\UFS12DL\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
